In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Ввод датасетов

In [2]:
df_2017 = pd.read_csv('archive/kaggle-survey-2017/multipleChoiceResponses.csv',
                      encoding='latin1', low_memory=False)

In [3]:
df_2017.head(1)

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorExperienceLevel,JobFactorDepartment,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity
0,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_2018 = pd.read_csv('archive/kaggle-survey-2018/multipleChoiceResponses.csv',
                      encoding='latin1', low_memory=False)

In [5]:
df_2018.head(1)

,Time from Start to Finish (seconds),Q1,Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,...,Q49_OTHER_TEXT,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_4,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,Q50_OTHER_TEXT
0,Duration (in seconds),What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,What is your age (# years)?,In which country do you currently reside?,What is the highest level of formal education ...,Which best describes your undergraduate major?...,Select the title most similar to your current ...,Select the title most similar to your current ...,In what industry is your current employer/cont...,...,What tools and methods do you use to make your...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...


In [6]:
df_2019 = pd.read_csv('archive/kaggle-survey-2019/multiple_choice_responses.csv',
                      encoding='latin1', low_memory=False)

In [7]:
df_2019.head(1)

,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,...,Q34_Part_4,Q34_Part_5,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,Select the title most similar to your current ...,What is the size of the company where you are ...,Approximately how many individuals are respons...,...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...


In [8]:
df_2020 = pd.read_csv('archive/kaggle-survey-2020/kaggle_survey_2020_responses.csv',
                      encoding='latin1', low_memory=False)

In [9]:
df_2020.head(1)

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q35_B_Part_2,Q35_B_Part_3,Q35_B_Part_4,Q35_B_Part_5,Q35_B_Part_6,Q35_B_Part_7,Q35_B_Part_8,Q35_B_Part_9,Q35_B_Part_10,Q35_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,...,"In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor..."


In [10]:
df_2021 = pd.read_csv('archive/kaggle-survey-2021/kaggle_survey_2021_responses.csv',
                      encoding='latin1', low_memory=False)

In [11]:
df_2021.head(1)

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q38_B_Part_3,Q38_B_Part_4,Q38_B_Part_5,Q38_B_Part_6,Q38_B_Part_7,Q38_B_Part_8,Q38_B_Part_9,Q38_B_Part_10,Q38_B_Part_11,Q38_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,...,"In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor..."


In [12]:
df_2022 = pd.read_csv('archive/kaggle-survey-2022/kaggle_survey_2022_responses.csv',
                      encoding='latin1', low_memory=False)

In [13]:
df_2022.head(1)

,ï»¿Duration (in seconds),Q2,Q3,Q4,Q5,Q6_1,Q6_2,Q6_3,Q6_4,Q6_5,...,Q44_3,Q44_4,Q44_5,Q44_6,Q44_7,Q44_8,Q44_9,Q44_10,Q44_11,Q44_12
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,"Are you currently a student? (high school, uni...",On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...


In [14]:
df_2025 = pd.read_csv('archive/kaggle-survey-2025/multipleChoiceResponses.csv',
                      encoding='latin1', low_memory=False)

In [15]:
df_2025.head(1)

,Time from Start to Finish (seconds),Q1,Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,...,Q49_OTHER_TEXT,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_4,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,Q50_OTHER_TEXT
0,Duration (in seconds),What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,What is your age (# years)?,In which country do you currently reside?,What is the highest level of formal education ...,Which best describes your undergraduate major?...,Select the title most similar to your current ...,Select the title most similar to your current ...,In what industry is your current employer/cont...,...,What tools and methods do you use to make your...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...,What barriers prevent you from making your wor...


# Column mapping

In [16]:
datasets = {
        '2018': df_2018, 
        '2019': df_2019,
        '2020': df_2020,
        '2021': df_2021,
        '2022': df_2022,
        '2025': df_2025
    }

In [17]:
def normalize_df(df):
    name2questions = df.iloc[0].to_list()
    df.columns = name2questions
    df.drop(0, inplace=True)

    return df

In [18]:
for key, df in datasets.items():
    df = normalize_df(df)

In [19]:
def auto_create_mapping(columns_list):
    mapping = {}
    
    for col in columns_list:
        simplified = simplify_column_name(col)
        mapping[col] = simplified
    
    return mapping

def simplify_column_name(column_name):
    name = column_name.lower()
    
    patterns_to_remove = [
        r'which of the following',
        r'select all that apply',
        r'selected choice',
        r'other - text',
        r'approximate \$usd',
        r'include text response',
        r'in the last 5 years',
        r'at work or school',
        r'what is your',
        r'how many',
        r'does your',
        r'select any',
        r'approximate what percent',
        r'how long',
        r'for how many',
        r'do you consider',
        r'what specific',
        r'what programming',
        r'what machine learning',
        r'what data visualization',
        r'of the choices that you selected in the previous question',
        r'which specific',
        r'what is the type',
        r'where do you',
        r'during a typical',
        r'what percentage',
        r'on which online platforms',
        r'who/what are your',
        r'how do you perceive',
        r'which better demonstrates',
        r'what metrics',
        r'what do you find most difficult',
        r'in what circumstances',
        r'what methods',
        r'what tools and methods',
        r'what barriers'
    ]
    
    for pattern in patterns_to_remove:
        name = re.sub(pattern, '', name)
    
    name = re.sub(r'[^a-z0-9\s]', ' ', name)
    name = re.sub(r'\s+', '_', name.strip())
    
    if len(name) > 50:
        keywords = ['gender', 'age', 'country', 'education', 'major', 'job', 'title', 
                   'industry', 'experience', 'compensation', 'tool', 'language', 'framework',
                   'ide', 'notebook', 'cloud', 'database', 'ml', 'ai', 'data', 'viz',
                   'time', 'year', 'skill', 'challenge', 'method', 'platform']
        
        for keyword in keywords:
            if keyword in name:
                name = keyword + '_' + name.split('_')[-1]
                break
    
    return name

with open('output.txt', 'r', encoding='utf-8') as f:
    content = f.read()

columns_2018 = eval(content)
    
auto_mapping = auto_create_mapping(columns_2018)

# Перевод столбцов в единый формат

In [20]:
import pandas as pd
import re
from difflib import SequenceMatcher
from typing import Dict, List, Tuple

def find_best_match(column_name: str, mapping_dict: Dict[str, str], threshold: float = 0.6) -> Tuple[str, str, float]:
    """
    Находит лучший match для столбца из маппинга на основе сходства названий.
    
    Parameters:
    -----------
    column_name : str
        Название столбца для которого ищем соответствие
    mapping_dict : dict
        Словарь маппинга {оригинальное_имя: нормализованное_имя}
    threshold : float
        Порог сходства (0-1), ниже которого соответствие не считается
        
    Returns:
    --------
    tuple: (лучшее_совпадение, нормализованное_имя, оценка_сходства)
    """
    column_lower = column_name.lower()
    best_match = None
    best_score = 0
    best_normalized = None
    
    for original_col, normalized_col in mapping_dict.items():
        original_lower = original_col.lower()
        
        # Вычисляем сходство
        similarity = SequenceMatcher(None, column_lower, original_lower).ratio()
        
        # Проверяем ключевые слова для улучшения точности
        if similarity > best_score:
            best_score = similarity
            best_match = original_col
            best_normalized = normalized_col
    
    # Если сходство выше порога, возвращаем результат
    if best_score >= threshold:
        return best_match, best_normalized, best_score
    
    return None, None, best_score

def smart_column_mapping(df_columns: List[str], mapping_dict: Dict[str, str], 
                        threshold: float = 0.6) -> Dict[str, str]:
    """
    Умный маппинг столбцов на основе сходства названий.
    
    Parameters:
    -----------
    df_columns : list
        Список названий столбцов для маппинга
    mapping_dict : dict
        Словарь маппинга {оригинальное_имя: нормализованное_имя}
    threshold : float
        Порог сходства для принятия решения
        
    Returns:
    --------
    dict: {оригинальное_имя_столбца: нормализованное_имя}
    """
    result_mapping = {}
    matches_info = []
    
    for col in df_columns:
        best_match, normalized_name, score = find_best_match(col, mapping_dict, threshold)
        
        if best_match:
            result_mapping[col] = normalized_name
            matches_info.append({
                'original': col,
                'matched_to': best_match,
                'normalized': normalized_name,
                'similarity': f"{score:.3f}"
            })
        else:
            matches_info.append({
                'original': col,
                'matched_to': 'NO_MATCH',
                'normalized': 'UNKNOWN',
                'similarity': f"{score:.3f}"
            })
    
    # Создаем DataFrame с информацией о маппинге
    mapping_df = pd.DataFrame(matches_info)
    print("Результаты маппинга:")
    print(mapping_df.to_string(index=False))
    
    return result_mapping

def apply_smart_mapping_to_datasets(datasets: Dict[str, pd.DataFrame], 
                                   base_mapping: Dict[str, str],
                                   threshold: float = 0.6) -> Dict[str, pd.DataFrame]:
    """
    Применяет умный маппинг ко всем датасетам.
    """
    standardized_datasets = {}
    
    for name, df in datasets.items():
        print(f"\n{'='*50}")
        print(f"Обрабатываю {name}...")        
        # Находим умный маппинг для этого датасета
        smart_mapping = smart_column_mapping(df.columns.tolist(), base_mapping, threshold)
        
        # Применяем маппинг
        df_std = df.rename(columns=smart_mapping)
        
        # Удаляем столбцы, для которых не нашли соответствия
        columns_to_drop = [col for col in df_std.columns if col not in smart_mapping.values()]
        if columns_to_drop:
            print(f"Удаляю {len(columns_to_drop)} столбцов без маппинга")
            df_std = df_std.drop(columns=columns_to_drop)
        
        # Добавляем год опроса
        year_match = re.search(r'\d{4}', name)
        year = year_match.group() if year_match else 'unknown'
        df_std['survey_year'] = year
        
        print(f"Результат: {df_std.shape[1]} столбцов")
        standardized_datasets[name] = df_std
    
    return standardized_datasets

# Дополнительные функции для улучшения точности
def enhance_matching(column_name: str, mapping_dict: Dict[str, str]) -> str:
    """
    Улучшенное сопоставление с использованием ключевых слов.
    """
    column_lower = column_name.lower()
    
    # Ключевые слова и их приоритетные соответствия
    keyword_patterns = {
        'gender': 'gender',
        'age': 'age', 
        'country': 'country',
        'education': 'formal_education',
        'major': 'undergraduate_major',
        'job': 'current_job_title',
        'title': 'current_job_title',
        'employer': 'employer_industry',
        'compensation': 'compensation_amount',
        'salary': 'compensation_amount',
        'experience': 'years_experience',
        'language': 'programming_language',
        'tool': 'work_tools',
        'machine learning': 'ml_skills',
        'learning': 'learning_platforms',
        'time': 'learning_time',
        'skill': 'skill_importance',
        'challenge': 'work_challenges'
    }
    
    for keyword, mapped_value in keyword_patterns.items():
        if keyword in column_lower:
            # Ищем оригинальное название в маппинге
            for orig, norm in mapping_dict.items():
                if norm == mapped_value:
                    return orig, mapped_value, 0.9
    
    return find_best_match(column_name, mapping_dict)

datasets = {
    '2018': df_2018, 
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
    '2025': df_2025
}
    
# Применяем умный маппинг
standardized_datasets = apply_smart_mapping_to_datasets(
    datasets, auto_mapping, threshold=0.5
)

print("\n" + "="*60)
print("ИТОГИ СТАНДАРТИЗАЦИИ:")
print("="*60)

for name, df in standardized_datasets.items():
    print(f"{name}: {df.shape[1]} столбцов")
    print(f"  Общие столбцы: {list(df.columns[:5])}...")


Обрабатываю 2018...
Результаты маппинга:
                                                                                                                                                                                                                                 original                                                                                                                                                                                                                                matched_to                                                                                                                                                     normalized similarity
                                                                                                                                                                                                                    Duration (in seconds)                                                                                       

# Объединение дубликатов

In [51]:
import tqdm

In [52]:
def efficient_merge_duplicates(df):
    """
    Эффективное объединение дубликатов без фрагментации
    """
    unique_columns = df.columns.unique()
    merged_data = {}
    
    for col in tqdm.tqdm(unique_columns):
        duplicate_cols = [c for c in df.columns if c == col]
        
        if len(duplicate_cols) == 1:
            # Просто копируем столбец
            merged_data[col] = df[col]
        else:
            # Объединяем дубликаты
            duplicate_df = df[duplicate_cols]
            
            # Создаем объединенный столбец
            merged_col = duplicate_df.apply(
                lambda x: x.dropna().iloc[0] if x.dropna().any() else np.nan, 
                axis=1
            )
            merged_data[col] = merged_col
    
    result_df = pd.DataFrame(merged_data, index=df.index)
    return result_df

In [59]:
normilized_datasets = standardized_datasets.copy()

In [60]:
normilized_datasets['2018'] = efficient_merge_duplicates(normilized_datasets['2018'])
normilized_datasets['2019'] = efficient_merge_duplicates(normilized_datasets['2019'])
normilized_datasets['2020'] = efficient_merge_duplicates(normilized_datasets['2020'])
normilized_datasets['2021'] = efficient_merge_duplicates(normilized_datasets['2021'])
normilized_datasets['2022'] = efficient_merge_duplicates(normilized_datasets['2022'])
normilized_datasets['2025'] = efficient_merge_duplicates(normilized_datasets['2025'])

100%|█████████████████████████████████████████| 339/339 [00:27<00:00, 12.47it/s]


# Создание датасета

In [61]:
cols_df_2018 = set(normilized_datasets['2018'].columns)
cols_df_2019 = set(normilized_datasets['2019'].columns)
cols_df_2020 = set(normilized_datasets['2020'].columns)
cols_df_2021 = set(normilized_datasets['2021'].columns)
cols_df_2022 = set(normilized_datasets['2022'].columns)
cols_df_2025 = set(normilized_datasets['2025'].columns)

In [62]:
common_cols = list(cols_df_2018.intersection(cols_df_2019, cols_df_2020, cols_df_2021, cols_df_2022, cols_df_2025))

In [65]:
normilized_datasets['2018'] = normilized_datasets['2018'][common_cols]
normilized_datasets['2019'] = normilized_datasets['2019'][common_cols]
normilized_datasets['2020'] = normilized_datasets['2020'][common_cols]
normilized_datasets['2021'] = normilized_datasets['2021'][common_cols]
normilized_datasets['2022'] = normilized_datasets['2022'][common_cols]
normilized_datasets['2025'] = normilized_datasets['2025'][common_cols]

In [66]:
combined_df = pd.concat(normilized_datasets.values(), ignore_index=True)

combined_df.to_csv('kaggle_survey_2018-2025.csv', index=False)